## HHW Lab 3 : Combinations Lab
In this notebook, we'll combine what you learned from the previous notebooks and start comparing the GNSS time series to the hydrologic measurements we have. Feel free to refer back to the previous notebooks for a bit of helpful code.

Goals for this notebook:
* Explore different methods of comparison
* Compare the GNSS vertical time series to multiple hydrologic datasets
* Create a water balance to estimate hydrologic storage
* Compare hydrologic storage to GNSS vertical time series 




For this, we've broken you up into groups and assigned different watersheds. At the end of the combinations lab, you and your team will put together a short presentation of your findings and how the vertical GNSS in your watershed compare to hydrologic measurements. 

Each watershed is classified by USGS as a HUC8 watershed and is roughly the same size. If you're interested in more info about hydrologic unit codes (HUC) or identifiers for other watersheds, you can get more info here: https://water.usgs.gov/GIS/huc.html.

If you were assigned the Idaho watershed (Camas Creek, HUC8), you'll be working with GNSS station P350, Snotel station 830, and USGS gage 13141500. The area of this watershed is 1779.73 sq km. 

If you were assigned the Sierra watershed (Upper Yuba Feather), you'll be working with GNSS station P144, Snotel station 428, and USGS gauge 11413000. The area of this watershed is 3483.36 sq km.

If you were assigned the CO watershed (Roaring Fork), you'll be working with GNSS station P728, Snotel station 737, and USGS gauge 09085000 - the most downstream gage that you'll use in part 4. USGS gage 09081000, gage 09076300, gage 09073400, gage 09073300, and gage 09072550 also exist along this river. The area of this watershed is 3767.35 sq km.

Everyone will work on one snow-dominated watershed (above) AND the same rain-dominated watershed (Russian River). For the Russian River, you'll be working with GNSS station P192, and USGS gauge 11467000 - the most downstream gage that you'll use in part 4. USGS gage 11464000, gage 11463000, gage 11462500, and gage 11461000 also exist along this river. The area of this watershed is 3845.99 sq km.

### PART 1: GNSS Data
Let's start with the GNSS data. You'll import the GNSS data, the NTOL and NTAL data, which I've already combined for you and in called combo21.pnum.txt. Go ahead and remove the NTAL/NTOL signal and cut the timeseries. 

In [2]:
##Import the libraries you'll likely need
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

##This is what you'll cut your time series to, GNSS and hydrologic
startdate = pd.to_datetime('10/1/2008', format = '%m/%d/%Y')
enddate = pd.to_datetime('9/29/2021', format = '%m/%d/%Y')

##Import the libraries you'll likely need
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

gps = pd.read_csv("GNSS_data/????_UNR.txt", header = None, 
                   skiprows = 1, names = ['Date', 'east', 'north','vertical', 'e_std','n_std','v_std'])
gps.Date = pd.to_datetime(gps.Date)
gps = gps.set_index('Date')
gps = gps[startdate:enddate]

##add in the atmospheric loading
atmos = pd.read_csv("GNSS_data/????.txt", skiprows = 1, header = None, 
                    names = ['a_n','a_e', 'a_v','Date'])
#Convert to datetime
atmos.Date = pd.to_datetime(atmos.Date)
#Convert from m to mm  
atmos.a_v = atmos.a_v * 1000
atmos = atmos.set_index('Date')
atmos = atmos.sort_values(by='Date')
    
full = pd.concat([gps3, atmos], axis=1)
full['new_v'] = full.vertical - full.a_v


FileNotFoundError: [Errno 2] No such file or directory: 'GNSS_data/????_UNR.txt'

Next, we'll fit and remove a linear trend. Just the linear trend since we want to keep the annual hydrologic signal in at this point but remove the long term tectonic signal. Again, I recommend plotting to make sure everything looks a-ok. If there is a large offset in your data, you can cut your time series to avoid it (or if you cruised through the GNSS notebook go ahead and fit and correct the offset). 

In [ ]:
full['t']=full.index.map(dt.datetime.toordinal)
full = full.dropna()

verticalseries = lin_remove(full.new_v, full.v_std, full.t) 
full.new_v.update(verticalseries)

Go ahead and fit the harmonic to the data and calculate what months the min and max occur - you can just estimate. **Don't remove it!** We'll just use the timing information later. 

In [ ]:
### Harmonic function
# y(t) = y_0 +vt_i + SUM(from k=1) a_k*sin(w_k * t_i) + b_k*cos(w_k*t_i)
# G*m = d style

from __future__ import division
import numpy as np

# Ingest 1 component of the timse seris, omega, time in ordinal 
def harm_fit(tseries, omega, dtime, semi):
    #Observations
    dv = np.array(tseries)
    if (semi == False): #just the annual
        #G matrix
        #Start with an empty matrix
        Gv = np.zeros((len(dv), 4))
        #Make the columns to be filled
        c1v = np.ones((len(dv)))
        c2v = np.array(dtime)
        c3v = np.sin(c2v*omega)
        c4v = np.cos(c2v*omega)
        #Fill the G matrix
        Gv[:,0] = c1v
        Gv[:,1] = c2v
        Gv[:,2] = c3v
        Gv[:,3] = c4v
        #Solve using Least Squares
        mv = np.linalg.solve(np.dot(np.transpose(Gv), Gv),np.dot(np.transpose(Gv),dv))
        # Full harmonic :
        y_v = mv[0] + mv[1]*dtime + mv[2]*np.sin(omega*dtime) + mv[3]*np.cos(omega*dtime)
    else: #annual + semi annual
        omega2 = 2*omega
        #G matrix
        #Start with an empty matrix
        G2 = np.zeros((len(dv), 6))
        #Make the columns to be filled
        c1 = np.ones((len(dv)))
        c2 = np.array(dtime)
        c3 = np.sin(c2*omega)
        c4 = np.cos(c2*omega)
        c5 = np.sin(c2*omega2)
        c6 = np.cos(c2*omega2)
        #Fill the G matrix
        G2[:,0] = c1
        G2[:,1] = c2
        G2[:,2] = c3
        G2[:,3] = c4
        G2[:,4] = c5
        G2[:,5] = c6
        #Solve using Least Squares
        m2 = np.linalg.solve(np.dot(np.transpose(G2), G2),np.dot(np.transpose(G2),dv))
        # Full harmonic :
        y_v = m2[0] + m2[1]*dtime + m2[2]*np.sin(omega*dtime) + m2[3]*np.cos(omega*dtime) + m2[4]*np.sin(omega2*dtime) +m2[5]*np.cos(omega2*dtime)
    return y_v, m2


In [ ]:
harmfit, m = harm_fit(full.new_v, 2*np.pi/365.25, full.t, True)

In [ ]:
#calculate when the harmonic reaches its min and max


Earlier, we were focused on daily time series. One method that is often used in GNSS processing (for hydrology or for other purposes) is applying a temporal smoothing to the data. This bumps up the signal to noise ratio of the GNSS data. With pandas, a simple moving mean or median is easy to do. We'll be using the "rolling function": https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

As with everything, there are many ways to do the same thing. If you're feeling confident in your python skills you can go ahead and figure out how to write a weighted rolling mean or median, incorporating the daily uncertainties. 

Go ahead and mess around with a few different temporal lengths - plotting them over the actual vertical time series to see how they perform. Try a mean and a median. 

In [ ]:
#plot your rolling means with the data until you determine the best one
full['rmed'] = full.new_v.rolling(??).median()
full['rmean'] = full.new_v.rolling(??).mean()

Great, now that you have a little feel for how the different options might work, we're going to be using a 30 day rolling median for the rest of this. The reason we are using a median over a mean is because a median is less sensitive to outliers. And even though you've removed most with your previous function there might be a few more that were missed. However, other scientists might prefer a mean. And if you feel strongly, then use a mean. Go ahead and fit a 30 day rolling median. For the rest of the notebook, we'll be using this 30 day rolling median so if I refer to the gps data this is what I'm talking about. 

Before we dive into the hydrologic data, let's check the GNSS data by plotting it. 

**Q1: Based on the GNSS data, describe the hydrologic signals that you see. Be specific.**

### PART 2: Hydrologic Data
Now let's retrieve the hydrologic data from Hydroshare for rain and snow dominated sites. For each snow-dominated site, you should find SWE from the Snotel station and streamflow from the USGS gage. You'll also find precipitation and reference evapotranspiration (ET) cut to the boundaries of the watershed with each measurement in every pixel within the watershed summed into one daily value from the Gridded Surface Meteorological dataset (Gridmet). For the rain-dominated site, you should find streamflow from the USGS gage. You'll also find precipitation and reference evapotranspiration (ET) summed over the watershed from Gridmet. Go ahead and check out the Gridmet website to make sure it makes sense where those values come from: https://www.climatologylab.org/gridmet.html 


Read those datasets into python using pandas. Be sure to track the units of each measurement. Personally, I like to add the units to the end of the column name like precip_mm, for example.    

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

#Read in hydrologic data
filename = 'CamasCreek_HydroData.csv'
Camas = pd.read_csv(filename, header = [0])
Camas['Date'] = pd.to_datetime(Camas['Date'], infer_datetime_format=True)
Camas.set_index(Camas['Date'], inplace = True)
Camas.dropna()

filename = 'RoaringFork_HydroData.csv'
RF = pd.read_csv(filename, header = [0])
RF['Date'] = pd.to_datetime(RF['Date'], infer_datetime_format=True)
RF.set_index(RF['Date'], inplace = True)
RF.dropna()

filename = 'YubaFeather_HydroData.csv'
YF = pd.read_csv(filename, header = [0])
YF['Date'] = pd.to_datetime(YF['Date'], infer_datetime_format=True)
YF.set_index(YF['Date'], inplace = True)
YF.dropna()

filename = 'RussianRiver_HydroData.csv'
RR = pd.read_csv(filename, header = [0])
RR['Date'] = pd.to_datetime(RR['Date'], infer_datetime_format=True)
RR.set_index(RR['Date'], inplace = True)
RR.dropna()

In [13]:
Camas['year'] = pd.DatetimeIndex(Camas['Date']).year
Camas['month'] = pd.DatetimeIndex(Camas['Date']).month
Camas['wateryear'] = Camas['year']
    
for i in range(len(Camas.Date)):
    if Camas.month[i] > 9:
        Camas.wateryear[i] = Camas.year[i] + 1

RF['year'] = pd.DatetimeIndex(RF['Date']).year
RF['month'] = pd.DatetimeIndex(RF['Date']).month
RF['wateryear'] = RF['year']
    
for i in range(len(RF.Date)):
    if RF.month[i] > 9:
        RF.wateryear[i] = RF.year[i] + 1
        
YF['year'] = pd.DatetimeIndex(YF['Date']).year
YF['month'] = pd.DatetimeIndex(YF['Date']).month
YF['wateryear'] = YF['year']
    
for i in range(len(YF.Date)):
    if YF.month[i] > 9:
        YF.wateryear[i] = YF.year[i] + 1
        
RR['year'] = pd.DatetimeIndex(RR['Date']).year
RR['month'] = pd.DatetimeIndex(RR['Date']).month
RR['wateryear'] = RR['year']
    
for i in range(len(RR.Date)):
    if RR.month[i] > 9:
        RR.wateryear[i] = RR.year[i] + 1
print('WY columns added')

C:\Users\aliss\AppData\Local\Temp\ipykernel_29580\1732074040.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Camas.wateryear[i] = Camas.year[i] + 1
C:\Users\aliss\AppData\Local\Temp\ipykernel_29580\1732074040.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RF.wateryear[i] = RF.year[i] + 1
C:\Users\aliss\AppData\Local\Temp\ipykernel_29580\1732074040.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  YF.wateryear[i] = YF.year[

WY columns added


**Q2: What is the resolution of gridmet? What data is available from gridmet?**

**Q3: What is the citation for Gridmet data?**

Go ahead and plot each time series to get an idea of how the datasets change over time. 

In [ ]:
#Plot hydrologic time series from your watershed

**Q4: Comment on any longer-term temporal changes that you see. Can you explain what may be causing these?** 

**Q5: Describe the datasets across one full WY. Do you notice a relationship between the fluxes (as in, as x decreases y begins to increase)?** 

Find the max SWE and streamflow and the dates of those peaks for every WY in your snow-dominated site and find the date associated with the max value. 

df_max_Q = max(df.variable) 
df_max_Q_date = df[['variable']].idxmax()

In [ ]:
#Calculate max 

**Q6: How do the SWE and streamflow max dates compare for your snow dominated watershed? What is the lag time between them (one max date - the other max date) and are those lags consistent across WYs within the whole time series?** 

**Q7: How do the dates of max streamflow and precip compare? Do you expect the lag to be larger or smaller in rain-dominated watersheds and why do you think that?** 

**Q8: For a few WYs, compare the timing of max streamflow between your snow and rain dominated watersheds. Go back to your earlier description of temporal changes (see Q4). Do these values support your earlier qualitative analysis?** 

Go ahead and compare the locations of all of the watersheds provided by creating a map using pyGMT. Feel free to refer back to the hydro notebook section 5 for helpful code to create this map. 

In [ ]:
#Create a map

**Q9: Based on the location of these sites, can you make any generalizations about the distribution of water across the western US?** 

### PART 3: GNSS and Hydrologic Comparisons
Let's dive into some comparisions of GNSS and hydrologic time series. We'll begin with the basics. Go ahead and plot the GPS vertical signal and and the Snotel SWE observation on the same plot. 

**Q10: Describe how the two signals compare. Be detailed here. Go ahead and zoom in temporally as well as comparing the big picture (e.g., specific water years vs the longer term trends)**

Now, let's quantatively compare the two signals. We'll be using a Pearson's correlation coefficient (sometimes referred to pearson's R) which is a measure of linear correlation. For a more detailed explanation: https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/. 

It's a standard stastical test to test how two datasets relate (linearly in this case). This is just one option - there are so many statistical tests that can be used. Python has a ton of options to perform a pearson's correlation -- numpy has corrcoeff, scipy stats has pearsons r, or you can write your own if you feel zesty. 

eg:

*variable = df.variable.corr(df2.variable2)*

or

*np.corrcoeff(df1.variable1, df2.variable2)*

Go head and calculate the pearson's R for the gps vertical signal and snotel swe. 


**Q11: Does this value make sense? If values closer to -1 and 1 indicate stronger correlation, how does this compare? What might make this correlation stronger?**

Another common statistical test is a cross correlation which compares two time series at a series of lagged/shifted positions. In other words, it's a measure of similiarity between two time series as a function of the lag of one series relative to the other. Here's a link with a more detailed explanation and a visual example: https://robosub.eecs.wsu.edu/wiki/ee/hydrophones/start or https://en.wikipedia.org/wiki/Cross-correlation. A cross correlation allows us to see if there is a time delay between the two series. Before reading the rest of the notebook, I want you to think about how this statistical test could be used in the context of hydrogeodesy, hydrology or geodesy in general. 

**Q12: Brainstorm how a cross correlation could be used? What about other uses for a Pearsons' correlation? Any other statistical test that you think could be useful?**

One thing you need to be careful of when running cross correlations is that it typically removes data on the end, shortening the time series that is being shifted, resulting in a smaller amount of data being compared as you increase the lag. If you're time series is long, this typically isn't a huge deal but remains something to be aware of. There are also functions that will loop the time series back on itself, as well.

As when performing any stastical test, you should be thinking about what you're trying to look into because **correlation does not equal causation**. I love this website about spurious correlations: https://www.tylervigen.com/spurious-correlations. It's always good to keep in mind the possible physical mechanism that could lead to correlation. Above, we just correlated SWE to GPS vertical postion. 

**Q13: What mechanism would cause SWE to be correlated to vertical position? (Not a trick question)**

For cross correlations you should be thinking the same thing. The sanity check of: does this make sense? It will also help you set up the problem. When you are using a cross correlation function you can define the max time delay you want to investigate, e.g., 10 days, 20 days, the whole time series length. You want to keep in mind what the mechanism could be that would produce that time delay. Unless there is a reason for it, you probably don't want to just automatically use the full time series length since that is a lot of calculations. 

Let's start with correlating SWE and GPS vertical position. 

**Q14: What do you think is the time delay that would produce the highest correlation? What do you think is the maximum time lag you should use? Give me justification for this.**

Within python there are many ways to perform a cross correlation including a few built in functions such as statsmodels ccf, or matplotlibs xcorr, or writing a small loop using one of the correlation coefficient functions (of which there are tons already written online). Go ahead and perform a cross correlation. Also, don't forget that you've applied a 30 day smoothing that can sometimes cause signals to be "shmeared" into the surrounding days. If you think this might be what is happening with your specific station go ahead and run the cross correlation without the smoothing too. 

Before doing this, you'll want to merge the GPS dataframe and the hydro dataset. 

*combined_df = pd.merge([df1,df2], axis = 1)*

And, also remove any NaNs using df = df.dropna()


eg

*plt.xcorr(df.variable, df.variable2, maxlags=numoflags)*


import statsmodels.tsa.stattools as smt

*smt.ccf(df.variable1, df.variable2)*



**Q15: Are you correct? Does this make sense? Feel free to ask a neighbor or us if you're confused.** 

While that might not have been the most interesting cross correlation to perform sometimes it's good to check if the data is behaving how you might expect (or maybe you didn't expect that, which is totally fine!, because now you've learned something. In my opinion, noodling around with the data is the best way to gain an understanding of what is going on). 

You don't have any snow measurements to compare to GNSS in your rain-dominated watershed (Russian River). Instead, compare daily precipitation to GNSS vertical deflection and move through all the same analysis that you just did in your snow-dominated watersheds.  

**Q16: How do your correlations of precip and vertical displacement compare to those of SWE and vertical displacement? Does this fit your expectation? Why or why not?**

**Q17: How does the time delay compare between rain and snow dominated watersheds? Can you explain why you are seeing these time delays?** 

So far, we have compared vertical deflection to the main inputs to our system. Let's move onto another cross correlation and go ahead and compare to a well-known output instead. We're going to compare the GPS vertical position to the streamflow in your watershed. If you have more then 1 gage in your system, compare it to both. But before you begin:

**Q18: What do you think you should set as your maximum lag?**

Great, go ahead and perform the cross correlation. 

**Q19: As always, does this make sense? Explain what might be the reason for this? If you had more then one gage, were they the same? Does your answer make sense?**  If you are starting to feel like this is tedious, I'm trying to force you all into good habits. Sometimes its easy to just run all these statistical tests or compute these mathematical models and trust whatever comes out of them but I want all of you to sanity check the answers - in this short course but also moving forward in whatever research you end up doing. Ok, I'll get off my soap box. 

### PART 4: Estimating hydrologic storage
You just saw that GNSS vertical displacements are somewhat related to system inputs and outputs. Now let's explore the relationship between those with a simple mass balance. We're going to estimate hydrologic storage using the following equation:
**dS/dt = Inputs - Outputs**

**Q20: What are the inputs to our system?**

**Q21: What about the outputs? Keep in mind that there is one more output we have not explored yet.**

**Q22: Write out your new dS/dt equation for each watershed. Keep in mind that the change in storage from the day before (t-1) will impact the daily storage for the current day (t).**

We also need to think about defining the boundaries of our system. We will use the watershed as our lateral bounds. The data provided to you has been cut to the watershed. 

What about the vertical bounds? For this simple estimate, we are excluding groundwater flow. There is no specific depth of the vertical boundary, but rather the water balance does not extend into the saturated zone of groundwater. We will not account for groundwater loss or gain because those constraints can be quite a bit more complex and challenging to find, especially in mountain systems.   

Another important thing to think about when estimating hydrologic storage is the units of our inputs and outputs. For your water balance, you'll want everything in terms of cubic meters/day. Remember back to our earlier discussion of daily fluxes. 

**Q23: What are the units of your hydrologic datasets? Streamflow? Precipitation? ET? SWE?**  

**Q24: What do you need to convert depth to volume? Hint: For precip and ET, remember back to Q2 about the resolution of Gridmet because we actually summed the measurement from each pixel. For SWE, we are making a very large assumption by assuming the SWE measured at our Snotel station is representative of the whole watershed so look back at our initial description of the watersheds.**  

Because we are calculating a daily change in storage, go ahead and convert everything to cubic meters/day. Hint: first, make sure you get everything into meters or m3/sec for streamflow. Then convert from depth to volume and account for the time component. 

Now, go ahead and estimate the daily change in hydrologic storage for your snow-dominated and rain-dominated watersheds using the datasets provided. Keep in mind that your equations for dS/dt will be slightly different for the two types of watersheds. Also, remember that snow-dominated watersheds can still be influenced by rainfall. Because SWE measures the volume and the mass  of the snowpack (to calculate density), liquid precipitation should be accounted for by SWE measurements during the snowpack; however, it will need to be addressed in the absense of SWE. 

A few other important hints: you will have to set an arbitrary start value for dS/dt for this calculation. Because there are many different scenarios to consider (if t == 0, if t > 0), you'll need to use a for loop for this calculation. In the snow-dominated waterhsed, if and elif statments will be needed to account for precip in the absence of SWE. Feel free to discuss with neighbors and us! If you are not familiar with for loops and if statements, we're happy to help you through this. It can be a bit frustrating at first. 

The avg2_v is the vertical of the GPS. 

In [1]:
for i in range(len(df.avg2_v)):
    if i == 0 :
        df.Storage_m3[i] = 0
    else :
        # df.Storage_m3[i] = df.Storage_m3[i-1] + df.precip_m3[i] + df.swe_m3[i] - df.ET_m3[i] - df.Q_m3[i]
        if df.swe_m3[i] > 0 :
            df.Storage_m3[i] = df.Storage_m3[i-1] + df.swe_m3[i] - df.ET_m3[i] - df.Q_m3[i]
        elif df.swe_m3[i] == 0 :
            df.Storage_m3[i] = df.Storage_m3[i-1] + df.precip_m3[i] - df.ET_m3[i] - df.Q_m3[i]


NameError: name 'df' is not defined

Go ahead and plot up your time series of hydrologic storage.

**Q25: Do your estimates of hydrologic storage follow the temporal trends that you expected to see? Explain.**  

It is very likely that they are trending strongly in one direction. What do you think might be causing this? 


Hopefully you guessed that reference ET is swamping the other fluxes. Let's chat briefly about ET. Remember when we were learning all about how the met station instruments worked and what they measured. You should have noticed that the Climavue (and nearly all met stations) are equipped to measure everything needed to calculate ET (typically using the Penman-Monteith equation but there are lots of options. Feel free to dig into those, if you're interested.) Unfortunately, actual ET (what is actually evaporated and transpired from the surface) is difficult to constrain at scales larger than point measurements because it depends on the met conditions, the amount of available water, and the surface type (i.e., plant type or water body). Therefore, we rely on estimates of what could evaporate and transpire at larger scales - either potential ET or reference ET. In this case, Gridmet calculates reference ET assuming that the land surface is covered with grass (with a fairly well constrained transpiration rate). That leads to an overestimation of ET, but at least it gives us somewhere to start from and gives a pretty good idea of how ET should change temporally.      

Brainstorm ideas to address this. Personally, I tend to set a constraint on my water balance that does not allow dS/dt to drop below zero. If this is what you are seeing in your time series of dS/dt, go ahead and address this issue by adding one more if statement to your for loop that calculates dS/dt (if S < 0) -- we did it already, see below. Compare your watershed storage again. 

In [ ]:
for i in range(len(df.avg2_v)):
    if i == 0 :
        df.Storage_m3[i] = 0
    else :
        # df.Storage_m3[i] = df.Storage_m3[i-1] + df.precip_m3[i] + df.swe_m3[i] - df.ET_m3[i] - df.Q_m3[i]
        if df.swe_m3[i] > 0 :
            df.Storage_m3[i] = df.Storage_m3[i-1] + df.swe_m3[i] - df.ET_m3[i] - df.Q_m3[i]
        elif df.swe_m3[i] == 0 :
            df.Storage_m3[i] = df.Storage_m3[i-1] + df.precip_m3[i] - df.ET_m3[i] - df.Q_m3[i]
    if df.Storage_m3[i] < 0 :
        df.Storage_m3[i] = 0

**Q26: Now, do you see any initial differences in the estimated hydrologic storage of your two watersheds?**

Finally, let's see how our estimates of hydrologic storage compare to GNSS vertical displacement. I find it helpful to compute relative values of storage and displacement using df.variable/max(df.variable) and compare the relative magnitudes rather than the absolute magnitudes because of the large difference in absolute values. Go ahead and calculate the relative magnitude of dS/dt and vertical displacement.

Now compute pearson correlation coefficients like you did earlier. Plot up your time series using shared x axes. 

**Q27: How does the relationship between estimated hydrologic storage and vertical displacement compare to the relationships between your hydrologic fluxes (i.e., inputs and outputs) and vertical displacement? Is this what you expected? Is it the same across both watersheds?**

**Q28: Is there a time lag? Go ahead and compute the cross correlation between storage and vertical component of GNSS - still using the relative magnitudes**

**Be sure to keep in mind the assumptions that went into our hydrologic storage estimates. Think about how they could impact the correlations.**

### PART 5: Putting it all together



Now, as a group, we want you to put a little presentation of your findings together for the class since you all had different watersheds. At a minimum, we want a map of both of your watersheds with the location of the GNSS station, Snotel station, and USGS stream gages and any other relevant things. Also, some of your interesting findings (but make sure you include the calculated storage vs vertical GNSS). We want a few minute presentation from each group 5-10 minutes (short and stress-free). 

### PART 6: Additional components for our speedy coders

Did you finish and still have a ton of time left over? If yes, continue on. If not, no worries you did everything we wanted you to get through. 


Alrighty, speedy coders. You have a few options and you can do either or both, depending on your interest. 

**OPTION 1**: Check out the LoadDef notebooks. LoadDef ingests load grids and calculates the response due to that load - e.g., it predicts the response at a given pixel due to a load (among other things). There are many applications relevant to this workshop including but not limited to: predicting surface displacements at GNSS stations due to GRACE  observations OR predicitng surface displacements due to a hydrologic model etc etc. It's currently a forward model program (though inverse modeling is in the works as well). Here's the full paper: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018EA000462 

The documentation for LoadDef is great and Hilary has provided some notebooks for anyone that wants to get started with the program. If this is of interest to you, go ahead and check those notebooks out. Later today, Matthew will also be briefly talking about LoadDef and inversions. 

**OPTION 2:** We talked very briefly about increasing signal to noise ratios. Another way to increase signal to noise ratios is to use multiple stations and combine the data - creating a network mean or median. If this is of interest to you, continue on. But honestly, LoadDef is way more interesting, in my opinion. 

Ok, option 2ers. I hope you've already done the LoadDef tutorial because I do honestly think it's way cooler. But here's what you'll do for option 2. Since you're advance coders you'll get way less prompting in this but feel free to ask us questions if you need help. 

Go ahead and pick a watershed from this notebook. Locate your station on the UNR NGL map view: http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html 

Thinking about your watershed boundary, go ahead and pick some stations within or nearby your watershed boundary (at least 2 more additional stations). I typically like 10 + but for this notebook that might just be a lot unless you write a nifty little for loop (hint hint). With your additional stations in hand, go ahead and download the data within the "common mode" folder. These stations have already been corrected for NTAL & NTOL. I suggest plotting each of your new stations to make sure there is no anomalous data or an unaccounted for offset. There are a variety of ways to calculate a network mean (or common mode) and they can be used in a variety of ways. For this purpose, we'll calculate a network mean or network median. Using all your stations, calculate the daily mean or median position. (Pandas makes this easy if you use datetime like we've been suggesting). I also like to plot this mean/median over all the stations' positions to see if it makes sense and is doing what I think it is. Once you have your network mean, compare it to the storage estimate for that watershed. And then go through the questions again - did it improve your estimates? Worsen it? Can you think of why or why this happened? Feel free to include this in your presentation if you made it this far. 

There are a bunch of ways to calculate a common mode. If you're interested in estimating the common mode to filter out longer wavelength signals this is one option (calculating the network mean for a much larger area than the one you are interested in and removing it from the time series) or using a statistical approach such as a PCA or ICA. 

If you're interested in common mode and other applications in geodesy check out some of these papers: https://link.springer.com/article/10.1007/s00190-020-01466-5 OR https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/97JB01378 OR https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005JB003806 and here's a plug for my old paper that uses a similar method to what I just did. Making a case for using a network mean to fill in gaps in GRACE data: https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2018WR023289 

